# Few-Shot Templates


Few-shot prompting của LangChain cung cấp một framework mạnh mẽ để hướng dẫn các language model tạo ra các output chất lượng cao bằng cách cung cấp các ví dụ được lựa chọn cẩn thận. Kỹ thuật này giảm thiểu nhu cầu fine-tuning model rộng rãi, đồng thời đảm bảo kết quả chính xác, nhận biết ngữ cảnh trong nhiều use case đa dạng.

**Few-shot prompt templates**: Định nghĩa cấu trúc và định dạng của các prompt bằng cách nhúng các ví dụ minh họa, hướng dẫn model tạo ra các output nhất quán.

**Example selection strategies**: Lựa chọn động các ví dụ phù hợp nhất cho một query nhất định, tăng cường sự hiểu biết ngữ cảnh và độ chính xác phản hồi của model.

**Chroma vector store**: Một utility mạnh mẽ để lưu trữ và truy xuất các ví dụ dựa trên semantic similarity, cho phép xây dựng prompt có khả năng mở rộng và hiệu quả.


## Environment setup

In [8]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(override=True, dotenv_path="../.env")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

# os.getenv("OPENAI_API_KEY")


## FewShotPromptTemplate

**Few-shot prompting** là một kỹ thuật mạnh mẽ hướng dẫn các mô hình ngôn ngữ tạo ra kết quả chính xác và phù hợp với ngữ cảnh bằng cách cung cấp một tập hợp nhỏ các ví dụ được thiết kế cẩn thận. **LangChain's FewShotPromptTemplate** đơn giản hóa quá trình này, cho phép người dùng xây dựng các prompt linh hoạt và có thể tái sử dụng cho nhiều nhiệm vụ khác nhau như trả lời câu hỏi, tóm tắt và sửa lỗi.

**Thiết kế Few-Shot Prompts**

* Xác định các ví dụ minh họa cấu trúc và kiểu dáng mong muốn của đầu ra.
* Đảm bảo các ví dụ bao gồm các trường hợp đặc biệt (edge cases) để cải thiện sự hiểu biết và hiệu suất của mô hình.

**Dynamic Example Selection**

* Sử dụng các kỹ thuật tương đồng ngữ nghĩa (semantic similarity) hoặc tìm kiếm dựa trên vector (vector-based search) để chọn các ví dụ phù hợp nhất cho một truy vấn.

**Tích hợp Few-Shot Prompts**

* Kết hợp các template prompt với các mô hình ngôn ngữ để tạo ra các chuỗi mạnh mẽ để tạo phản hồi.

### Ví dụ về FewShotPromptTemplate

**FewShotPromptTemplate** cho phép bạn cung cấp cho một mô hình ngôn ngữ một tập hợp nhỏ các ví dụ minh họa cấu trúc và định dạng mong muốn của đầu ra. Bằng cách tận dụng các ví dụ này, mô hình có thể hiểu rõ hơn ngữ cảnh và tạo ra các phản hồi chính xác hơn cho các truy vấn mới. Kỹ thuật này đặc biệt hữu ích cho các nhiệm vụ như trả lời câu hỏi, tóm tắt hoặc tạo ra các đầu ra có cấu trúc.

Dưới đây, chúng ta định nghĩa một vài ví dụ để giúp mô hình trả lời câu hỏi hiệu quả hơn bằng cách chia chúng thành các bước trung gian. Sau đó, chúng ta sử dụng **FewShotPromptTemplate** để định dạng prompt một cách động dựa trên truy vấn.


In [3]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

# Define examples for the few-shot prompt
examples = [
    {
        "question": "Who lived longer, Steve Jobs or Einstein?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: At what age did Steve Jobs die?
Intermediate Answer: Steve Jobs died at the age of 56.
Additional Question: At what age did Einstein die?
Intermediate Answer: Einstein died at the age of 76.
The final answer is: Einstein
""",
    },
    {
        "question": "When was the founder of Naver born?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is the founder of Naver?
Intermediate Answer: Naver was founded by Lee Hae-jin.
Additional Question: When was Lee Hae-jin born?
Intermediate Answer: Lee Hae-jin was born on June 22, 1967.
The final answer is: June 22, 1967
""",
    },
    {
        "question": "Who was the reigning king when Yulgok Yi's mother was born?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is Yulgok Yi's mother?
Intermediate Answer: Yulgok Yi's mother is Shin Saimdang.
Additional Question: When was Shin Saimdang born?
Intermediate Answer: Shin Saimdang was born in 1504.
Additional Question: Who was the king of Joseon in 1504?
Intermediate Answer: The king of Joseon in 1504 was Yeonsangun.
The final answer is: Yeonsangun
""",
    },
    {
        "question": "Are the directors of Oldboy and Parasite from the same country?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is the director of Oldboy?
Intermediate Answer: The director of Oldboy is Park Chan-wook.
Additional Question: Which country is Park Chan-wook from?
Intermediate Answer: Park Chan-wook is from South Korea.
Additional Question: Who is the director of Parasite?
Intermediate Answer: The director of Parasite is Bong Joon-ho.
Additional Question: Which country is Bong Joon-ho from?
Intermediate Answer: Bong Joon-ho is from South Korea.
The final answer is: Yes
""",
    },
]

In [4]:
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)
print(example_prompt.format(**examples[0]))

Question:
Who lived longer, Steve Jobs or Einstein?
Answer:
Does this question require additional questions: Yes.
Additional Question: At what age did Steve Jobs die?
Intermediate Answer: Steve Jobs died at the age of 56.
Additional Question: At what age did Einstein die?
Intermediate Answer: Einstein died at the age of 76.
The final answer is: Einstein



In [5]:
# Initialize the FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

# Example question
question = "How old was Bill Gates when Google was founded?"

# Generate the final prompt
final_prompt = few_shot_prompt.format(question=question)
print(final_prompt)

Question:
Who lived longer, Steve Jobs or Einstein?
Answer:
Does this question require additional questions: Yes.
Additional Question: At what age did Steve Jobs die?
Intermediate Answer: Steve Jobs died at the age of 56.
Additional Question: At what age did Einstein die?
Intermediate Answer: Einstein died at the age of 76.
The final answer is: Einstein


Question:
When was the founder of Naver born?
Answer:
Does this question require additional questions: Yes.
Additional Question: Who is the founder of Naver?
Intermediate Answer: Naver was founded by Lee Hae-jin.
Additional Question: When was Lee Hae-jin born?
Intermediate Answer: Lee Hae-jin was born on June 22, 1967.
The final answer is: June 22, 1967


Question:
Who was the reigning king when Yulgok Yi's mother was born?
Answer:
Does this question require additional questions: Yes.
Additional Question: Who is Yulgok Yi's mother?
Intermediate Answer: Yulgok Yi's mother is Shin Saimdang.
Additional Question: When was Shin Saimdang bo

## Dynamic Example Selection with Chroma

Đôi khi chúng ta cần trải qua nhiều bước suy nghĩ để đánh giá một câu hỏi. Việc chia nhỏ câu hỏi thành các bước và hướng tới câu trả lời mong muốn có thể dẫn đến phản hồi chất lượng tốt hơn. **Chroma** cung cấp một cách hiệu quả để lưu trữ và truy xuất các ví dụ dựa trên sự tương đồng ngữ nghĩa, cho phép lựa chọn ví dụ động trong quy trình làm việc.

**Khởi tạo Embedding và Vector Store**

Sử dụng **OpenAIEmbeddings** để nhúng các ví dụ.
Lưu trữ các embedding trong một **Chroma** vector store để truy xuất hiệu quả.

**Lưu trữ Ví dụ**

Các ví dụ được lưu trữ cùng với nội dung và siêu dữ liệu của chúng.
Siêu dữ liệu có thể bao gồm các chi tiết như câu hỏi và câu trả lời, được sử dụng để xử lý thêm sau khi truy xuất.

**Tìm kiếm Tương đồng**

Truy vấn vector store để truy xuất các ví dụ tương tự nhất dựa trên đầu vào.
Cho phép tạo prompt động nhận biết ngữ cảnh.

**Nhập các gói cần thiết và tạo một vector store.**


In [10]:
from langchain_openai.embeddings.base import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

embedding = OpenAIEmbeddings()
chroma = Chroma(persist_directory="example_selector", embedding_function=embedding)

In [ ]:
examples = [
    {
        "question": "Who lived longer, Steve Jobs or Einstein?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: At what age did Steve Jobs die?
Intermediate Answer: Steve Jobs died at the age of 56.
Additional Question: At what age did Einstein die?
Intermediate Answer: Einstein died at the age of 76.
The final answer is: Einstein
""",
    },
    {
        "question": "When was the founder of Google born?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is the founder of Google?
Intermediate Answer: Google was founded by Larry Page and Sergey Brin.
Additional Question: When was Larry Page born?
Intermediate Answer: Larry Page was born on March 26, 1973.
Additional Question: When was Sergey Brin born?
Intermediate Answer: Sergey Brin was born on August 21, 1973.
The final answer is: Larry Page was born on March 26, 1973, and Sergey Brin was born on August 21, 1973.
""",
    },
    {
        "question": "Who was the President when Donald Trump's mother was born?", 
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is Donald Trump's mother?
Intermediate Answer: Donald Trump's mother is Mary Anne MacLeod Trump.
Additional Question: When was Mary Anne MacLeod Trump born?
Intermediate Answer: Mary Anne MacLeod Trump was born on May 10, 1912.
Additional Question: Who was the U.S. President in 1912?
Intermediate Answer: William Howard Taft was President in 1912.
The final answer is: William Howard Taft
""",
    },
    {
        "question": "Are the directors of Oldboy and Parasite from the same country?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is the director of Oldboy?
Intermediate Answer: The director of Oldboy is Park Chan-wook.
Additional Question: Which country is Park Chan-wook from?
Intermediate Answer: Park Chan-wook is from South Korea.
Additional Question: Who is the director of Parasite?
Intermediate Answer: The director of Parasite is Bong Joon-ho.
Additional Question: Which country is Bong Joon-ho from?
Intermediate Answer: Bong Joon-ho is from South Korea.
The final answer is: Yes
""",
    },
]

Create a vector store and define the `DynamicFewShotLearning` template.

In [13]:
# Add examples to vector store
texts = [example['question'] for example in examples]
chroma.add_texts(texts=texts, metadatas=examples)

['ecd21ae3-0d91-4e0a-89ea-57eeb9dee760',
 '53086468-998b-4198-8193-2416f1f71d52',
 '4d1527f0-5a55-43c3-bc25-2735d66da49d',
 '1c8a357e-9250-487a-b856-bb58d4abf71b']

In [14]:
# Setup example selector
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=chroma,
    k = 1 # number of examples to select
)

In [15]:
# Define Few-Shot Prompt Template
example_prompt_template = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}\n"
)
prompt = FewShotPromptTemplate(
    example_prompt= example_prompt_template,
    example_selector=example_selector,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

In [16]:
query = {"question": "How old was Elon Musk when he made Paypal?"}
formatted_prompt = prompt.format(**query)

In [17]:
# Print the constructed prompt
print("Constructed Prompt:\n")
print(formatted_prompt)

Constructed Prompt:

Question:
When was the founder of Google born?
Answer:
Does this question require additional questions: Yes.
Additional Question: Who is the founder of Google?
Intermediate Answer: Google was founded by Larry Page and Sergey Brin.
Additional Question: When was Larry Page born?
Intermediate Answer: Larry Page was born on March 26, 1973.
Additional Question: When was Sergey Brin born?
Intermediate Answer: Sergey Brin was born on August 21, 1973.
The final answer is: Larry Page was born on March 26, 1973, and Sergey Brin was born on August 21, 1973.



Question:
How old was Elon Musk when he made Paypal?
Answer:


## `FewShotChatMessagePromptTemplate`

Việc tạo ra các prompt cho từng tình huống là một nhiệm vụ phức tạp và tẻ nhạt. `FewShotChatMessagePromptTemplate` tận dụng phương pháp học few-shot (few-shot learning) để tạo ra các prompt dựa trên chat một cách động bằng cách kết hợp các ví dụ liên quan với định dạng có cấu trúc. Phương pháp này đặc biệt hiệu quả cho các nhiệm vụ như tóm tắt (summarization), tạo biên bản cuộc họp (meeting minute creation) hoặc xử lý tài liệu (document processing).



In [18]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai.embeddings.base import OpenAIEmbeddings
from langchain_chroma import Chroma

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Initialize the Vector DB
chroma = Chroma(persist_directory="fewshot_chat", embedding_function=embeddings)

# Define examples for few-shot prompting : 
examples = [
    {
        "instruction": "You are an expert in writing meeting minutes. Please write meeting minutes based on the given information",
        "input": "On December 25, 2023, XYZ Company's marketing strategy meeting began at 3 PM. The meeting was attended by Marketing Team Leader John Smith, Digital Marketing Manager Sarah Johnson, and Social Media Manager Mike Wilson. The main purpose of the meeting was to establish marketing strategies for the first half of 2024 and discuss ideas for new social media campaigns. Team Leader John Smith provided a brief overview of recent market trends, followed by presentations from each team member about strategic ideas in their respective fields.",
        "answer": """
Meeting Minutes: XYZ Company Marketing Strategy Meeting
Date: December 25, 2023
Location: XYZ Company Conference Room
Attendees: John Smith (Marketing Team Leader), Sarah Johnson (Digital Marketing Manager), Mike Wilson (Social Media Manager)

1. Opening
   - Meeting began with opening remarks by Team Leader John Smith.
   - Meeting purpose: Establish marketing strategies for H1 2024 and discuss new social media campaign ideas.

2. Market Trend Overview (John Smith)
   - Team Leader John Smith presented analysis of recent market trends.
   - Shared insights on consumer behavior changes and competitor strategies.

3. Digital Marketing Strategy (Sarah Johnson)
   - Sarah Johnson presented digital marketing strategy.
   - Focused on online advertising and SEO optimization methods.

4. Social Media Campaign (Mike Wilson)
   - Mike Wilson proposed ideas for new social media campaigns.
   - Explained plans for influencer marketing and content strategy.

5. Comprehensive Discussion
   - Team members shared and discussed ideas.
   - Discussed budget and resource allocation for each strategy.

6. Closing
   - Confirmed next meeting date and time.
   - Sarah Johnson assigned to compile and distribute meeting minutes.
""",
    },
    {
        "instruction": "You are a summarization expert. Please summarize the content based on the given information",
        "input": "This document is a 20-page report on 'Strategies for Sustainable Urban Development'. The report comprehensively covers the importance of sustainable urban development, current urbanization issues, and various strategies to make urban development sustainable. The report also introduces successful sustainable urban development cases from multiple countries and summarizes the lessons learned from these cases.",
        "answer": """
Document Summary: Strategy Report for Sustainable Urban Development

- Importance: Emphasizes the necessity of sustainable urban development and its social, economic, and environmental benefits.
- Current Issues: Analyzes major problems in current urbanization processes, such as environmental pollution, resource depletion, and increasing inequality.
- Strategies: Presents various strategies to achieve sustainable urban development. These include eco-friendly construction, public transportation improvements, energy efficiency enhancement, and strengthening community engagement.
- Case Studies: Introduces successful sustainable development cases from cities worldwide. For example, explains achievable strategies through cases like Copenhagen, Denmark and Yokohama, Japan.
- Lessons: Summarizes key lessons learned from these cases. Emphasized lessons include the importance of multi-faceted approaches, cooperation with local communities, and the need for long-term planning.

This report provides an in-depth analysis of how sustainable urban development can be realized in practical and effective forms.
""",
    },
    {
        "instruction": "You are a sentence correction expert. Please correct the following sentences",
        "input": "Our company is planning to introduce a new marketing strategy. Through this, communication with customers will become more effective.",
        "answer": "This company expects to improve customer communication more effectively by introducing a new marketing strategy.",
    },
]

In [19]:
# Add examples to the vector store
texts = [example["instruction"] + " " + example["input"] for example in examples]
metadatas = examples
chroma.add_texts(texts=texts, metadatas=metadatas)

# Example query
query = {
    "instruction": "Please write the meeting minutes",
    "input": "On December 26, 2023, the product development team of ABC Technology Company held a weekly progress meeting for a new mobile application project. The meeting was attended by Project Manager Choi Hyun-soo, Lead Developer Hwang Ji-yeon, and UI/UX Designer Kim Tae-young. The main purpose of the meeting was to review the current progress of the project and establish plans for upcoming milestones. Each team member provided updates on their respective areas of work, and the team set goals for the next week.",
}

# Perform similarity search
query_text = query["instruction"] + " " + query["input"]
results = chroma.similarity_search(query_text, k=1)
print(results)

[Document(id='2edb8391-4bf6-4f89-8b1e-91e5b2c757e2', metadata={'answer': '\nMeeting Minutes: XYZ Company Marketing Strategy Meeting\nDate: December 25, 2023\nLocation: XYZ Company Conference Room\nAttendees: John Smith (Marketing Team Leader), Sarah Johnson (Digital Marketing Manager), Mike Wilson (Social Media Manager)\n\n1. Opening\n   - Meeting began with opening remarks by Team Leader John Smith.\n   - Meeting purpose: Establish marketing strategies for H1 2024 and discuss new social media campaign ideas.\n\n2. Market Trend Overview (John Smith)\n   - Team Leader John Smith presented analysis of recent market trends.\n   - Shared insights on consumer behavior changes and competitor strategies.\n\n3. Digital Marketing Strategy (Sarah Johnson)\n   - Sarah Johnson presented digital marketing strategy.\n   - Focused on online advertising and SEO optimization methods.\n\n4. Social Media Campaign (Mike Wilson)\n   - Mike Wilson proposed ideas for new social media campaigns.\n   - Explain

In [20]:

# Print the most similar example
print(f"Most similar example to the input:\n{query_text}\n")
for result in results:
    print(f'Instruction:\n{result.metadata["instruction"]}')
    print(f'Input:\n{result.metadata["input"]}')
    print(f'Answer:\n{result.metadata["answer"]}')

Most similar example to the input:
Please write the meeting minutes On December 26, 2023, the product development team of ABC Technology Company held a weekly progress meeting for a new mobile application project. The meeting was attended by Project Manager Choi Hyun-soo, Lead Developer Hwang Ji-yeon, and UI/UX Designer Kim Tae-young. The main purpose of the meeting was to review the current progress of the project and establish plans for upcoming milestones. Each team member provided updates on their respective areas of work, and the team set goals for the next week.

Instruction:
You are an expert in writing meeting minutes. Please write meeting minutes based on the given information
Input:
On December 25, 2023, XYZ Company's marketing strategy meeting began at 3 PM. The meeting was attended by Marketing Team Leader John Smith, Digital Marketing Manager Sarah Johnson, and Social Media Manager Mike Wilson. The main purpose of the meeting was to establish marketing strategies for the f

In [21]:
# Create the final prompt template
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        (
            "human",
            "{instruction}:\n{input}",
        ),
    ]
)

# Combine the prompt and the best example
best_example = results[0].metadata
filled_prompt = final_prompt.format_messages(**best_example)

# Print the filled prompt
print("\nFilled Prompt:\n")
for message in filled_prompt:
    # Determine message type and extract content
    message_type = type(message).__name__  # e.g., SystemMessage, HumanMessage, AIMessage
    content = message.content
    print(f"{message_type}:\n{content}\n")


Filled Prompt:

SystemMessage:
You are a helpful assistant.

HumanMessage:
You are an expert in writing meeting minutes. Please write meeting minutes based on the given information:
On December 25, 2023, XYZ Company's marketing strategy meeting began at 3 PM. The meeting was attended by Marketing Team Leader John Smith, Digital Marketing Manager Sarah Johnson, and Social Media Manager Mike Wilson. The main purpose of the meeting was to establish marketing strategies for the first half of 2024 and discuss ideas for new social media campaigns. Team Leader John Smith provided a brief overview of recent market trends, followed by presentations from each team member about strategic ideas in their respective fields.



**Giải quyết các vấn đề Tìm kiếm Tương đồng trong Example Selector**

Khi tính toán sự tương đồng, cả **instruction** và **input** đều được sử dụng. Tuy nhiên, việc tìm kiếm chỉ dựa trên **instruction** không mang lại kết quả tương đồng chính xác.

Để giải quyết vấn đề này, một lớp tùy chỉnh (custom class) cho việc tính toán sự tương đồng được định nghĩa.

Dưới đây là một ví dụ về kết quả được truy xuất không chính xác.


In [23]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai.embeddings.base import OpenAIEmbeddings
from langchain_chroma import Chroma

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Initialize the Vector DB
chroma = Chroma(persist_directory="fewshot_chat", embedding_function=embeddings)

# Define examples for few-shot prompting
examples = [
    {
        "instruction": "You are an expert in writing meeting minutes. Please write meeting minutes based on the given information",
        "input": "On December 25, 2023, XYZ Company's marketing strategy meeting began at 3 PM. The meeting was attended by Marketing Team Leader Kim Su-jin, Digital Marketing Manager Park Ji-min, and Social Media Manager Lee Jun-ho. The main purpose of the meeting was to establish marketing strategies for the first half of 2024 and discuss ideas for new social media campaigns. Team Leader Kim Su-jin provided a brief overview of recent market trends, followed by presentations from each team member about strategic ideas in their respective fields.",
        "answer": """
Meeting Minutes: XYZ Company Marketing Strategy Meeting
Date: December 25, 2023
Location: XYZ Company Conference Room
Attendees: Kim Su-jin (Marketing Team Leader), Park Ji-min (Digital Marketing Manager), Lee Jun-ho (Social Media Manager)

1. Opening
   - Meeting began with opening remarks by Team Leader Kim Su-jin.
   - Meeting purpose: Establish marketing strategies for H1 2024 and discuss new social media campaign ideas.

2. Market Trend Overview (Kim Su-jin)
   - Team Leader Kim Su-jin presented analysis of recent market trends.
   - Shared insights on consumer behavior changes and competitor strategies.

3. Digital Marketing Strategy (Park Ji-min)
   - Park Ji-min presented digital marketing strategy.
   - Focused on online advertising and SEO optimization methods.

4. Social Media Campaign (Lee Jun-ho)
   - Lee Jun-ho proposed ideas for new social media campaigns.
   - Explained plans for influencer marketing and content strategy.

5. Comprehensive Discussion
   - Team members shared and discussed ideas.
   - Discussed budget and resource allocation for each strategy.

6. Closing
   - Confirmed next meeting date and time.
   - Park Ji-min assigned to compile and distribute meeting minutes.
""",
    },
    {
        "instruction": "You are a summarization expert. Please summarize the content based on the given information",
        "input": "This document is a 20-page report on 'Strategies for Sustainable Urban Development'. The report comprehensively covers the importance of sustainable urban development, current urbanization issues, and various strategies to make urban development sustainable. The report also introduces successful sustainable urban development cases from multiple countries and summarizes the lessons learned from these cases.",
        "answer": """
Document Summary: Strategy Report for Sustainable Urban Development

- Importance: Emphasizes the necessity of sustainable urban development and its social, economic, and environmental benefits.
- Current Issues: Analyzes major problems in current urbanization processes, such as environmental pollution, resource depletion, and increasing inequality.
- Strategies: Presents various strategies to achieve sustainable urban development. These include eco-friendly construction, public transportation improvements, energy efficiency enhancement, and strengthening community engagement.
- Case Studies: Introduces successful sustainable development cases from cities worldwide. For example, explains achievable strategies through cases like Copenhagen, Denmark and Yokohama, Japan.
- Lessons: Summarizes key lessons learned from these cases. Emphasized lessons include the importance of multi-faceted approaches, cooperation with local communities, and the need for long-term planning.

This report provides an in-depth analysis of how sustainable urban development can be realized in practical and effective forms.
""",
    },
    {
        "instruction": "You are a sentence correction expert. Please correct the following sentences",
        "input": "Our company is planning to introduce a new marketing strategy. Through this, communication with customers will become more effective.",
        "answer": "This company expects to improve customer communication more effectively by introducing a new marketing strategy.",
    },
]

In [24]:
# Add examples to the vector store
texts = [example["instruction"] + " " + example["input"] for example in examples]
metadatas = examples
chroma.add_texts(texts=texts, metadatas=metadatas)

['b465cc28-bc54-4785-a450-f8b530d946ea',
 '5626c7ee-4c96-4512-9e7b-e75804267a2a',
 'd1ddf1b8-ef7f-4dd5-afe4-0728639fb3f6']

In [25]:
# Use Chroma for example selection
def select_examples(query):
    query_text = query["instruction"] + " " + query.get("input", "")
    results = chroma.similarity_search(query_text, k=1)
    print("\n[select_examples Output]")
    print(f"Query Text: {query_text}")
    for i, result in enumerate(results, start=1):
        print(f"Result {i}:")
        print(f"Page Content: {result.page_content}")
        print(f"Metadata: {result.metadata}")
    return results

# Example selector using Chroma
def custom_selector(query):
    results = select_examples(query)
    selected_examples = [
        {
            "instruction": result.metadata["instruction"],
            "input": result.metadata["input"],
            "answer": result.metadata["answer"],
        }
        for result in results
    ]
    print("\n[custom_selector Output]")
    for i, example in enumerate(selected_examples, start=1):
        print(f"Selected Example {i}:")
        print(f"Instruction: {example['instruction']}")
        print(f"Input: {example['input']}")
        print(f"Answer: {example['answer']}")
    return selected_examples

# Example query to test
query = {
    "instruction": "Please write the meeting minutes",
    "input": "On December 26, 2023, the product development team of ABC Technology Company held a weekly progress meeting for a new mobile application project. The meeting was attended by Project Manager Choi Hyun-soo, Lead Developer Hwang Ji-yeon, and UI/UX Designer Kim Tae-young. The main purpose of the meeting was to review the current progress of the project and establish plans for upcoming milestones. Each team member provided updates on their respective areas of work, and the team set goals for the next week.",
}

# Test the functions
custom_selector(query)


[select_examples Output]
Query Text: Please write the meeting minutes On December 26, 2023, the product development team of ABC Technology Company held a weekly progress meeting for a new mobile application project. The meeting was attended by Project Manager Choi Hyun-soo, Lead Developer Hwang Ji-yeon, and UI/UX Designer Kim Tae-young. The main purpose of the meeting was to review the current progress of the project and establish plans for upcoming milestones. Each team member provided updates on their respective areas of work, and the team set goals for the next week.
Result 1:
Page Content: You are an expert in writing meeting minutes. Please write meeting minutes based on the given information On December 25, 2023, XYZ Company's marketing strategy meeting began at 3 PM. The meeting was attended by Marketing Team Leader Kim Su-jin, Digital Marketing Manager Park Ji-min, and Social Media Manager Lee Jun-ho. The main purpose of the meeting was to establish marketing strategies for th

[{'instruction': 'You are an expert in writing meeting minutes. Please write meeting minutes based on the given information',
  'input': "On December 25, 2023, XYZ Company's marketing strategy meeting began at 3 PM. The meeting was attended by Marketing Team Leader Kim Su-jin, Digital Marketing Manager Park Ji-min, and Social Media Manager Lee Jun-ho. The main purpose of the meeting was to establish marketing strategies for the first half of 2024 and discuss ideas for new social media campaigns. Team Leader Kim Su-jin provided a brief overview of recent market trends, followed by presentations from each team member about strategic ideas in their respective fields.",
  'answer': '\nMeeting Minutes: XYZ Company Marketing Strategy Meeting\nDate: December 25, 2023\nLocation: XYZ Company Conference Room\nAttendees: Kim Su-jin (Marketing Team Leader), Park Ji-min (Digital Marketing Manager), Lee Jun-ho (Social Media Manager)\n\n1. Opening\n   - Meeting began with opening remarks by Team Lead

In [26]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

# Define a new example prompt template with the custom selector
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

# Create a prompt template with selected examples
def create_fewshot_prompt(query):
    selected_examples = custom_selector(query)
    fewshot_prompt = ""
    for example in selected_examples:
        fewshot_prompt += example_prompt.format(
            instruction=example["instruction"],
            input=example["input"],
            answer=example["answer"],
        )
    return fewshot_prompt

In [27]:
# Create the final prompt for the chain
def create_final_prompt(query):
    fewshot_prompt = create_fewshot_prompt(query)
    final_prompt = (
        "You are a helpful professional assistant.\n\n" + fewshot_prompt + "\n\n"
        f"Human:\n{query['instruction']}:\n{query['input']}\nAssistant:"
    )
    return final_prompt

# Example queries
queries = [
    {
        "instruction": "Draft meeting minutes for the following discussion",
        "input": "On December 26, 2023, ABC Tech's product development team held their weekly progress meeting regarding the new mobile app project. Present were Project Manager John Davis, Lead Developer Emily Chen, and UI/UX Designer Michael Brown. The team reviewed current project milestones and established next steps. Each team member provided updates on their respective workstreams, and the team set deliverables for the upcoming week.",
    },
    {
        "instruction": "Please provide a summary of the following document",
        "input": "This comprehensive 30-page report titled 'Global Economic Outlook 2023' covers sustainable urban development trends, current urbanization challenges, and strategic approaches to sustainable city planning. The document includes case studies of successful urban development initiatives from various countries and key takeaways from these implementations.",
    },
    {
        "instruction": "Please review and correct these sentences",
        "input": "The company anticipates revenue growth this fiscal year. The new strategic initiatives are showing positive results.",
    },
]

# Process each query and print the responses
for query in queries:
    final_prompt = create_final_prompt(query)
    print("\nFinal Prompt:\n")
    print(final_prompt)
    response = llm.predict(final_prompt)
    print("\nModel Response:\n", response)
    print("\n---\n")


[select_examples Output]
Query Text: Draft meeting minutes for the following discussion On December 26, 2023, ABC Tech's product development team held their weekly progress meeting regarding the new mobile app project. Present were Project Manager John Davis, Lead Developer Emily Chen, and UI/UX Designer Michael Brown. The team reviewed current project milestones and established next steps. Each team member provided updates on their respective workstreams, and the team set deliverables for the upcoming week.
Result 1:
Page Content: You are an expert in writing meeting minutes. Please write meeting minutes based on the given information On December 25, 2023, XYZ Company's marketing strategy meeting began at 3 PM. The meeting was attended by Marketing Team Leader John Smith, Digital Marketing Manager Sarah Johnson, and Social Media Manager Mike Wilson. The main purpose of the meeting was to establish marketing strategies for the first half of 2024 and discuss ideas for new social media 

C:\Users\Dino\AppData\Local\Temp\ipykernel_7732\485481067.py:31: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(final_prompt)



Model Response:
 Meeting Minutes: ABC Tech Product Development Team Progress Meeting
Date: December 26, 2023
Location: ABC Tech Office
Attendees: John Davis (Project Manager), Emily Chen (Lead Developer), Michael Brown (UI/UX Designer)

1. Opening
   - Meeting commenced with Project Manager John Davis welcoming the team.
   - Meeting purpose: Review progress on new mobile app project and establish next steps.

2. Project Milestones Review
   - Team reviewed current project milestones.
   - Discussed any challenges faced and solutions implemented.

3. Workstream Updates
   - Emily Chen provided updates on development progress.
   - Michael Brown shared updates on UI/UX design enhancements.

4. Next Steps
   - Team discussed and set deliverables for the upcoming week.
   - Agreed on timelines and responsibilities for each task.

5. Comprehensive Discussion
   - Team members shared feedback and suggestions.
   - Discussed potential improvements and optimizations for the project.

6. Clos